In [1]:
import statsmodels.api as sm
from statsmodels.api import OLS, add_constant
from statsmodels.tsa.stattools import adfuller
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import random
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
#from arctic import Arctic
#from fs import open_fs
import os
from itertools import combinations
import json
import requests
import datetime
import time
from collections import deque
import csv
import ccxt
from pathlib import Path
import calendar

In [ ]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')
def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv

def write_to_csv(filename, exchange, data):
    p = Path("./data/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)
        #print(data)
        
def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)
   # print("Im Done with {0}".format(len(olhv)))


In [ ]:
#jupyter notebook --generate-config
#jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
#The notebook server will temporarily stop sending output to the client in order to avoid crashing it. 
#Applied on the notebook as a command prompt on Anaconda compiler.


In [ ]:
pwd   #Data will be stored inside the data folder of this notebook's directory.

In [ ]:
symbols = ['BTC','UNI','SUSHI','CRV','SOL','MANA','COMP','DOGE','BAT','MATIC','ENJ','XMR']

for symbol in symbols:
    scrape_candles_to_csv(f'{symbol}_USDT_5m.csv', 'binance', 3, f'{symbol}/USDT', '5m', '2021-05-1500:00:00Z', 1000)

In [ ]:
#To avoid notebook crashing symbols list has been 
symbols = ['LINK','BCH','XLM','ADA','BNB','LTC','ETH','AAVE','DOT','LINK','RUNE','YFI']

for symbol in symbols:
    scrape_candles_to_csv(f'{symbol}_USDT_5m.csv', 'binance', 3, f'{symbol}/USDT', '5m', '2021-05-1500:00:00Z', 1000)

In [2]:
#pwd
#Checking some of the symbols
df=pd.DataFrame()
symbols= ['ETH','BTC', 'BNB', 'SOL', 'AAVE', 'DOT', 'MATIC', 'LINK', 'RUNE','YFI']
for i in symbols:
    if(i==symbols[0]):
        df['Time']=pd.read_csv('C:\\Users\\91626\\ml\\project\\Quant Res\\Live code test\\data\\Binance\\'+i+'_USDT_5m.csv',index_col=False,names=['Time','Open','Close','Low','High'])['Time']
        
    df[i]=pd.read_csv('C:\\Users\\91626\\ml\\project\\Quant Res\\Live code test\\data\\Binance\\'+i+'_USDT_5m.csv',index_col=False,names=['Time','Open','Close','Low','High'])['Close']

C:\Users\91626\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [4]:
h=[]
for i in df['Time']:
    dt_string=str(i)
    print(i)
    format = "%Y-%m-%dT%H:%M:%S"
    h.append(datetime.datetime.strptime(dt_string, format))
df['Time']=h

In [ ]:
df.set_index('Time',inplace=True)

In [5]:
df.tail()

,Time,ETH,BTC,BNB,SOL,AAVE,DOT,MATIC,LINK,RUNE,YFI
117995,2022-10-12 21:20:00,1300.00,58951.57,271.1,31.20,71.7,6.23,0.8001,7.245,1.532,7868.0
117996,2022-10-12 21:25:00,1300.79,58880.86,271.1,31.20,71.7,6.23,0.8007,7.247,1.532,7866.0
117997,2022-10-12 21:30:00,1299.46,58899.81,271.0,31.25,71.8,6.23,0.7996,7.246,1.532,7866.0
117998,2022-10-12 21:35:00,1300.01,58919.85,271.2,31.30,71.9,6.23,0.7993,7.246,1.532,7859.0
117999,2022-10-12 21:40:00,1297.94,58913.34,270.9,31.29,71.8,6.23,0.7999,7.247,1.531,7848.0
